In [11]:
import tqdm
import urllib
import os.path
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup as bf


AMAZON_URL = "https://www.amazon.com/s?k={}&i=movies-tv"
AMAZON_URL_HEAD = "https://www.amazon.com"
HEADERS = ['User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36']

MOVIELENS_PATH = './'
OUTPUT_CSV = "./prices.csv"
DATABASE_PATH = "./database.db"

### Load movies.csv and links.csv

In [2]:
movies_df = pd.read_csv(MOVIELENS_PATH + 'movies.csv', dtype=str)
print(movies_df.head())

links_df = pd.read_csv(MOVIELENS_PATH + 'links.csv',dtype=str)
print(links_df.head())

  movieId                               title  \
0       1                    Toy Story (1995)   
1       2                      Jumanji (1995)   
2       3             Grumpier Old Men (1995)   
3       4            Waiting to Exhale (1995)   
4       5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
  movieId   imdbId tmdbId
0       1  0114709    862
1       2  0113497   8844
2       3  0113228  15602
3       4  0114885  31357
4       5  0113041  11862


### Join movies_df and links_df by movieId

In [3]:
merged_df = pd.merge(movies_df, links_df, on='movieId')
print(merged_df.head())

  movieId                               title  \
0       1                    Toy Story (1995)   
1       2                      Jumanji (1995)   
2       3             Grumpier Old Men (1995)   
3       4            Waiting to Exhale (1995)   
4       5  Father of the Bride Part II (1995)   

                                        genres   imdbId tmdbId  
0  Adventure|Animation|Children|Comedy|Fantasy  0114709    862  
1                   Adventure|Children|Fantasy  0113497   8844  
2                               Comedy|Romance  0113228  15602  
3                         Comedy|Drama|Romance  0114885  31357  
4                                       Comedy  0113041  11862  


### Create Database and Table

Table structure: movieId (int)，imdbId (varchar30), title (varchar50), pageUrl (varchar300), price (double)

In [4]:
database = sqlite3.connect(DATABASE_PATH)
cursor = database.cursor()
cursor.execute('create table if not exists prices (movieId integer primary key, imdbId varchar(30) NOT NULL, title varchar(50) NOT NULL, pageUrl varchar(300), price double)')
database.commit()

### Query exist tables

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Current table: ", cursor.fetchall())

Current table:  [('prices',)]


### Query table structure

In [6]:
cursor.execute("PRAGMA table_info(prices)")
print("Table prices: ", cursor.fetchall())

Table prices:  [(0, 'movieId', 'integer', 0, None, 1), (1, 'imdbId', 'varchar(30)', 1, None, 0), (2, 'title', 'varchar(50)', 1, None, 0), (3, 'pageUrl', 'varchar(300)', 0, None, 0), (4, 'price', 'double', 0, None, 0)]


### Insert movie data if not exist

In [7]:
row_num = merged_df.shape[0]
pbar = tqdm.trange(row_num, position=0, leave=True)
for i in pbar:
    pbar.set_description("Processing row %d" % i)
    row_item = merged_df.iloc[i]
    cursor.execute("SELECT * FROM prices WHERE movieId = ?;", (int(row_item['movieId']),))
    if len(cursor.fetchall()) == 0:
        cursor.execute("INSERT INTO prices (movieId, imdbId, title, pageUrl, price) VALUES (?, ?, ?, NULL, NULL)",
                       (int(row_item['movieId']), row_item['imdbId'], row_item['title']))
        database.commit()

Processing row 27277: 100%|██████████| 27278/27278 [00:46<00:00, 586.46it/s]


### Display first 5 data from database

In [8]:
cursor.execute("SELECT * FROM prices LIMIT 0, 10;")
print(cursor.fetchall())

[(1, '0114709', 'Toy Story (1995)', None, None), (2, '0113497', 'Jumanji (1995)', None, None), (3, '0113228', 'Grumpier Old Men (1995)', None, None), (4, '0114885', 'Waiting to Exhale (1995)', None, None), (5, '0113041', 'Father of the Bride Part II (1995)', None, None), (6, '0113277', 'Heat (1995)', None, None), (7, '0114319', 'Sabrina (1995)', None, None), (8, '0112302', 'Tom and Huck (1995)', None, None), (9, '0114576', 'Sudden Death (1995)', None, None), (10, '0113189', 'GoldenEye (1995)', None, None)]


In [9]:
def getPageUrl(title):
    url = AMAZON_URL.format(title.replace(" ", "+"))
    req = urllib.request.Request(url)
    req.add_header(HEADERS[0], HEADERS[1])
    html = urllib.request.urlopen(req)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def removeBrackets(title):
    return title[:-7] + " " + title[-5:-1]

In [12]:
getPageUrl(removeBrackets("Grumpier Old Men (1995)"))

'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619251071&s=movies-tv&sr=1-1'

In [ ]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
pbar = tqdm.tqdm(range(len(movie_list)))
for i in pbar:
    try:
        pbar.set_description("Processing row %d" % i)
        m = movie_list[i]
        movie_id = m[0]
        movie_title = m[2]
        page_url = m[3]
        if page_url is None:
            page_url = getPageUrl(removeBrackets(movie_title))
            cursor.execute("UPDATE prices SET pageUrl = ? WHERE movieId = ?;", (page_url, movie_id))
            database.commit()
    except UserAbort:
        break
    except KeyboardInterrupt:
        sys.exit()
        pass
    except Exception as e:
        print("Exception at row %d" % i)
        print(e)
        continue

Processing row 29:   0%|          | 28/27278 [00:56<14:48:08,  1.96s/it]

Exception at row 28


Processing row 42:   0%|          | 42/27278 [01:20<12:12:03,  1.61s/it]

Exception at row 41


Processing row 46:   0%|          | 46/27278 [01:26<10:15:49,  1.36s/it]

...

Processing row 49:   0%|          | 49/27278 [01:29<6:45:20,  1.12it/s] 

Exception at row 47
Exception at row 48


Processing row 50:   0%|          | 50/27278 [01:29<5:07:53,  1.47it/s]

Exception at row 49


Processing row 51:   0%|          | 51/27278 [01:31<8:29:28,  1.12s/it]

[(1, '0114709', 'Toy Story (1995)', 'https://www.amazon.com/Toy-Story-Tim-Allen/dp/B0094KTAEY/ref=sr_1_1?dchild=1&keywords=Toy+Story+1995&qid=1619250694&s=movies-tv&sr=1-1', None), (2, '0113497', 'Jumanji (1995)', 'https://www.amazon.com/Jumanji-Robin-Williams/dp/B00BZBQ4GW/ref=sr_1_1?dchild=1&keywords=Jumanji+1995&qid=1619250716&s=movies-tv&sr=1-1', None), (3, '0113228', 'Grumpier Old Men (1995)', 'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619250718&s=movies-tv&sr=1-1', None), (4, '0114885', 'Waiting to Exhale (1995)', 'https://www.amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_movies-tv_sr_pg1_1?ie=UTF8&adId=A04628882O8G5S747EVCD&url=%2FWaiting-Exhale-Heartbeats-Soulfood-Feature%2Fdp%2FB013LWX212%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3DWaiting%2Bto%2BExhale%2B1995%26qid%3D1619250720%26s%3Dmovies-tv%26sr%3D1-1-spons%26psc%3D1%26smid%3DAB1108BA04GDD&qualifier=1619250720&id=267466105838956

### Load data from database to Dataframe

In [ ]:
output_df = pd.DataFrame(columns=['movieId', 'title', 'pageUrl', 'price'])
cursor.execute("SELECT * FROM prices;")
pbar = tqdm.tqdm(cursor.fetchall(), position=0, leave=True)
for row in pbar:
    pbar.set_description("Processing %d" % 1)
    row_dict = {'movieId': row[0], 'title': row[1], 'pageUrl': row[2], 'price': row[3]}
    output_df = output_df.append(row_dict, ignore_index=True)

Processing 1:  77%|███████▋  | 20920/27278 [01:58<00:37, 170.03it/s]

### Write into CSV file

In [121]:
output_df.to_csv(OUTPUT_CSV, index=False)

In [24]:
'Jumanji (1995)'.

'Jumanji 1995'

In [2]:
!open .